# characterize political threads in subreddits

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import sqlite3
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

from utils import ROOTPATH, build_df_year, generate_index

In [3]:
df2012 = build_df_year(2012, with_text=False, with_time=True)
generate_index()

from utils import USERS, SUBREDDITS, user2index, index2user, subr2index, index2subr

building base dataframe for year 2012...
- fetching submissions for year 2012
- fetching comments for year 2012


In [3]:
df2012

,submission_id,author,subreddit,toxicity,body,comment_id,parent_id
0,ny6mv,jaki_cold,Anarchism,0.463043,"I think he means ""sex workers"".",NaN,NaN
1,nyddq,micahglasser,Anarchism,0.064921,This is Mutualism!,NaN,NaN
2,nyfti,im_not_a_troll,Anarchism,0.325869,Anarcho-communist children's book from 1901 (s...,NaN,NaN
3,nyghm,sfacets,Anarchism,0.089401,New subreddit - r/Cheatthesystem: Cheating the...,NaN,NaN
4,nyjdf,orxT1000,Anarchism,0.064297,28c3: A Brief History of Plutocracy,NaN,NaN
...,...,...,...,...,...,...,...
5568644,15p82u,baldylox,ShitPoliticsSays,0.574389,Dafuq did I just read?,c7ottc0,t3_15p82u
5568645,15qc8u,ShamAbram,ShitPoliticsSays,0.217722,"After intense research, I've found that /r/pol...",c7oug11,t3_15qc8u
5568646,15ojwl,Bazooko,ShitPoliticsSays,0.310894,Yup. It's not a difficult distinction to make....,c7ovra5,t1_c7ooh66
5568647,15qc8u,TheDeuceBaba,ShitPoliticsSays,0.440153,Anyone who posts in /r/politics is literally H...,c7owgr3,t1_c7oug11


## aggregate by submission

In [5]:
aggr_func = {"comment_id": lambda x: list(x)}
groupby_subm = df2012.groupby("submission_id").agg(aggr_func).reset_index()

In [6]:
aggr_subm

,submission_id,comment_id
0,10002y,[nan]
1,10003q,[nan]
2,1000ba,[nan]
3,1000ce,"[nan, c696q84, c696srf, c696ydr, c69788m, c697..."
4,1000en,"[nan, c696wf6, c6970rg, c697tt5, c698m6s, c698..."
...,...,...
469346,zzzsk,[nan]
469347,zzzte,"[nan, c69823f, c69btx6, c69e09r]"
469348,zzzus,"[nan, c696l2y, c69cr9s]"
469349,zzzw7,"[nan, c696olg, c696pd0, c696zp6, c6975ul, c69a..."
